## There is a small problem in codes at part 1.2

## To calculate magnetic field of each dipole from distance r,we use 
$\mathbf{B}(\mathbf{r}) = \nabla \times \mathbf{A} = \frac{\mu_0}{4\pi} \left[ \frac{3\mathbf{r}(\mathbf{m} \cdot \mathbf{r})}{r^5} - \frac{\mathbf{m}}{r^3} \right]$ (Equation 1)


The magnetization is often not listed as a material parameter for commercially available ferromagnetic materials. Instead, the parameter that is listed is *residual flux density* (or remanence), denoted $\mathbf{B_r}$. The formula needed in this case to calculate $\mathbf{m}$ in (units of A·m²) is:

$$
\mathbf{m} = \frac{1}{\mu_0} \mathbf{B_r} V      
$$ 
(Equation 2)


Where:
- $\mathbf{B_r}$ is the residual flux density, expressed in teslas.  
- $V$ is the volume of the magnet (in m³).  
- $\mu_0$ is the permeability of vacuum ($4\pi \times 10^{-7}$ H/m).
Retrieved from ---> https://en.wikipedia.org/wiki/Magnetic_moment


We will do our calculations for N52 grade magnets.

In [ ]:
import numpy as np
import math
import plotly.graph_objects as go
#Angle directions uses conventions on that page  --> https://en.wikipedia.org/wiki/Spherical_coordinate_system
#We ignore calculation of H field for reasonable approximation

def coordinate_calculator(radius,number_of_magnets): #Calculates the coordinates in 3D ----->Before Creating HallbachArray object,use this.
    #We will use it as an attribute to feed HalbachArray class 
    coords=[]
    
    
    
    increment=(2*math.pi)/number_of_magnets


    for i in range(number_of_magnets):
        coords.append([radius*np.cos(i*increment),radius*np.sin(i*increment),0])
    return np.array(coords)




class HalbachArray:  #Our array is placed on x-y plane,and the circular gap of array is in z axis.
    full_angle=2*math.pi  #Angle of full circle
    
    B_r=1.45 #Tesla// I took values from here --> https://www.stanfordmagnets.com/what-is-the-difference-between-n35-and-n52-magnets.html
    #B_r is the remenance 
    
    #Dimesions of magnet 0.01*0.01*0.01 meters
    
    def __init__(self,n_of_magnets,radius_of_array,dimensions_of_square_screen,resolution,theta_array,phi_array,coordinates): #Resolution is how much data points we have measured inside of the viewing screen of magnetic field.More precisely resolution is how many points per axis you want to see.
        self.n_of_magnets=n_of_magnets #n_of_magnets is the number of magnets in halbach array 
        self.radius_of_array=radius_of_array
        self.dimensions_of_square_screen=dimensions_of_square_screen #This is the place where we put magnetic bots (For our purpose we will see it as a viewing screen of magnetic field)
        self.resolution=resolution
        self.theta_array=np.deg2rad(np.array(theta_array)) #These are the orientation angles for each magnet we used in hallbach array.From zero'th magnet to nth magnet,we are entering each angle here.
        self.phi_array=np.deg2rad(np.array(phi_array)) #Angles will taken as degrees and converted here to radians for proper calculation
        self.coordinates=coordinates #We will take coordinates as np array from used function above. These are the coordinates of each magnet.
    def is_ok(self): #Check whether input parameters are correct
        if len(self.theta_array)==self.n_of_magnets and len(self.phi_array)==self.n_of_magnets:
            return True
        else:
    
            return False
    
    
   
    
    def coordinates_of_magnets(self): #Each magnet is called with its number.First magnet is numbered as zero.Additionally,first magnet is placed at angle zero in polar coordinates and remaining ones are placed in counter-clockwise direction properly.
        
        coordinates=[]
        angle_increment=(self.full_angle)/self.n_of_magnets
        for i in range(self.n_of_magnets):
            coordinates.append([self.radius_of_array*math.cos(0+i*angle_increment),self.radius_of_array*math.sin(0+i*angle_increment)])
        return np.array(coordinates) #The function returns coordinate arrays at the end
    
    

    
    
    def plot_magnetic_field_screen(self): #It will plot the magnetic field screen with seismic colormap #It will calculate and plot magnetic field via plotly 3D cone plot.
        #We will define m vector,which is the same as m in Equation 2
        m=[]
        Mu_zero=4*math.pi*10**(-7)
        Volume_of_magnet=0.01**3 #In m**3
        m_scalar=(1/Mu_zero)*(self.B_r)*Volume_of_magnet
        for i in range(len(self.theta_array)):
            m.append([m_scalar*np.sin(self.theta_array[i])*np.cos(self.phi_array[i]),    m_scalar*np.sin(self.theta_array[i])*np.sin(self.phi_array[i]) , m_scalar*np.cos(self.theta_array[i])    ])
           #m_y.append(m_scalar*np.sin(self.theta_array[i])*np.sin(self.phi_array[i]))
           #m_z.append(m_scalar*np.cos(self.theta_array[i]))
        #Now define m vector which is valid for each magnet,each 

        #Now we have calculated magnitudes of our direction vector r for our magnetic dipole,we are ready to dive in our magnetic field calculation and plotting
        m=np.array(m)



        #Now calculate the s vectors with respect to the origin.(These are places where we take measurement in viewing screen) 
        
        s_coords=[[[-((self.dimensions_of_square_screen)/2)+i*(self.dimensions_of_square_screen/(self.resolution-1)),((self.dimensions_of_square_screen)/2)-j*((self.dimensions_of_square_screen)/(self.resolution-1))    ,0] for j in range(self.resolution)] for i in range(self.resolution)]
            
        



        r_vectors=[] #By starting from 0'th magnet and point [0,0] ,we will calculate r vectors for each point and each magnet. 
        '''Structure [   
                        [
                            [r(00)0,r(00)1,r(00)2,......,r(00)n], ----> Position vectors array for point 00 ---> For B calculation at 00 all the vectors in that part are enough
                            [r(01)0,r(01)1,r(01)2,......,r(01)n], 
                            .....,
                            [r(0n)0,r(0n)1,r(0n)2,......,r(0n)n]                #All vectors in first row
                            
                            
                        ],



                        [



                            [r(10)0,r(10)1,r(10)2,......,r(10)n],
                            [r(11)0,r(11)1,r(11)2,......,r(11)n],
                            .....,
                            [r(1n)0,r(1n)1,r(1n)2,......,r(1n)n],




                        ]
                        .....
                        .....
                        .....
                        .....
                        .....
                        .....
                        [

                            [r(n0)0,r(n0)1,r(n0)2,......,r(n0)n],
                            [r(n1)0,r(n1)1,r(n1)2,......,r(n1)n],
                            .....,
                            [r(nn)0,r(nn)1,r(nn)2,......,r(nn)n]

                        ]
        
        
        
                     ]'''
        #Calculate r_vectors array r(xyn) n is naming number of n'th magnet
        #You need to scan s_coords array with two index
            #---> First index to scan s_coords
             #---> Second index to scan s_coords
                 #n is the index of magnets around the circle
                    #r_vectors.append(self.coordinates[n]-np.array([s_coords[i3][0],s_coords[i3][],s_coords[][]]))
        r_vectors=[[[s_coords[idx1][idx2]-self.coordinates[idx3] for idx1 in range(self.resolution)] for idx2 in range(self.resolution)] for idx3 in range(self.n_of_magnets)     ]






      
           
        

        #Each point in the screen is named with its coordinates [x,y].
        
        def calculate_B_field(m_,r_): # Takes m and r as numpy arrays 
            Mu_naught=4*math.pi*10**(-7)
            B = (Mu_naught/(4*math.pi))*(      (3*r_*(np.dot(m_,r_)))/(np.linalg.norm(r_)**5)     - m_/(np.linalg.norm(r_)**3)                           )
            return B
        #Now we are ready to calculate B field at each measurement
        B_at_screen=[]
        for a in range(self.resolution): #Here we will only calculate magnitudes of B vectors in each axis at each point in viewing screen
            for b in range(self.resolution):
                B_at_point=0 # It resets B_at_point variable when we move from one point to another in viewing screen.
                for c in range(self.n_of_magnets):
                    B_at_point += calculate_B_field(m[c],r_vectors[a][b][c]) #Superpose all the magnetic fields originated from all magnets for point [a,b] in viewing screen
                    B_at_screen.append(B_at_point) # For compatibility with positions array,we will convert it to same dimentional array with s_coords array.s_coords was the locations of each measurement point in the viewing screen.
        #Now hopefully our magnetic field at each point is ready

# We left at comparing compatibility of B_at_screen array with s_coords array. ----> In other words how to we use loops to scan both of them to both take magnitudes andstarting points of vectors                   





        

## Try to use Plotly

In [ ]:
import plotly.graph_objects as go










figure=go.Figure(data=go.Cone(x=[0],y=[0],z=[0],u=[3],v=[3],w=[3],sizemode="absolute",sizeref=2,anchor="tip"))
figure.update_layout(
      scene=dict(domain_x=[0, 1],
                 camera_eye=dict(x=-1.57, y=1.36, z=0.58)))
figure.show()





## I will continue to update the file...

In [ ]:
arr=[]
res=4
#One liner----
s_coords=[[[i,j,0] for j in range(res)] for i in range(res)]
print(s_coords)

In [ ]:
import numpy as np
s_coords=np.array([[[-((10)/2)+i*(10/(10-1)),((10)/2)-j*((10)/(10-1))    ,0] for j in range(10)] for i in range(10)])
print(s_coords.shape)

In [ ]:
array=np.array([[0,0],[0,1],[1,0],[1,1]])
array.reshape(-2,2)
print(array)

## m'leri Kontrol Et